In [15]:
import tensorflow as tf
from IPython.display import display
import pandas as pd
import seaborn as sns
import tensorflow.keras.backend as K
import numpy as np
import os
import gc
import glob
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import BatchNormalization, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
%matplotlib inline

In [2]:
def tf_reset_graph(model=None):
    tf.reset_default_graph()
    K.clear_session()
    gc.collect()

def tf_reset_callbacks(checkpoint=None, reduce_lr=None, early_stopping=None, tensorboard=None):
    checkpoint = None
    reduce_lr = None
    early_stopping = None
    tensorboard = None

def create_image_data_generator(target_size, rescale, path, batch_size, class_mode = 'categorical', shuffle=True, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True):

    data_generator = ImageDataGenerator(rescale=rescale,
                                        shear_range=shear_range,
                                        zoom_range=zoom_range,
                                        horizontal_flip=horizontal_flip)

    return data_generator.flow_from_directory(path,
                                       class_mode=class_mode,
                                       target_size=target_size,
                                       shuffle=shuffle,
                                       batch_size=batch_size)

In [3]:
image_ext = '.jpeg'
data_dir = 'dataset/'
test_dir = data_dir+'test/'
train_dir = data_dir+'train/'
val_dir = data_dir+'val/'
normals = 'NORMAL/'
infected = 'PNEUMONIA/'
training_normal_nos = len(glob.glob1(train_dir+normals,'*'+image_ext))
training_infected_nos = len(glob.glob1(train_dir+infected,'*'+image_ext))
training_nos = training_infected_nos+training_normal_nos
testing_normal_nos = len(glob.glob1(test_dir+normals,'*'+image_ext))
testing_infected_nos = len(glob.glob1(test_dir+infected,'*'+image_ext))
testing_nos = testing_infected_nos+testing_normal_nos
validation_normal_nos = len(glob.glob1(val_dir+normals,'*'+image_ext))
validation_infected_nos = len(glob.glob1(val_dir+infected,'*'+image_ext))
validation_nos = validation_infected_nos+validation_normal_nos

print('Total number of training images: ', training_nos)
print('Total number of testing images: ', testing_nos)
print('Total number of validation images: ', validation_nos)

Total number of training images:  5216
Total number of testing images:  624
Total number of validation images:  16


In [4]:
rescale_factor = 1./255
target_size = (150,150)
class_mode = 'categorical'
training_batch_size = int(training_nos/40)

train_data_generator = create_image_data_generator(target_size=target_size, 
                                                   rescale=rescale_factor, 
                                                   path=train_dir, 
                                                   batch_size=training_batch_size)

test_data_generator = create_image_data_generator(target_size=target_size,
                                                 rescale=rescale_factor,
                                                 path=test_dir,
                                                 batch_size=testing_nos)

val_data_generator = create_image_data_generator(target_size=target_size,
                                                 rescale=rescale_factor,
                                                 path=val_dir,
                                                 batch_size=validation_nos)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [39]:
tf_reset_callbacks()
tf_reset_graph()
base = 'bin'
model_dir = os.path.join(base,'models')
log_dir = os.path.join(base,'logs')
model_file = model_dir + "{epoch:02d}-val_acc-{val_acc:.2f}-val_loss-{val_loss:.2f}.hdf5"

checkpoint = ModelCheckpoint(
    model_file, 
    monitor='val_acc', 
    save_best_only=True)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1,
    restore_best_weights=True)

tensorboard = TensorBoard(
    log_dir=log_dir,
    batch_size=training_batch_size,
    update_freq = 'batch')

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=5,
    cooldown=2,
    min_lr=1e-10,
    verbose=1)

#callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard]
callbacks = [checkpoint, reduce_lr, tensorboard]

In [40]:
def create_model():
    inception_model = InceptionV3(weights='imagenet', include_top=False, input_shape = target_size+(3,))
    x = inception_model.output
    x = BatchNormalization()(x)
    x = Dense(100, activation='sigmoid')(x)
    x = Flatten()(x)
    predictions = Dense(2,activation='softmax')(x)
    model = Model(inputs = inception_model.input, outputs=predictions)
    for layer in inception_model.layers:
        layer.trainable = False
    return model

In [42]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [41]:
steps_per_epoch = len(train_data_generator)
validation_steps = len(val_data_generator)
classes = train_data_generator.classes
class_weights = class_weight.compute_class_weight('balanced', np.unique(classes), classes)

optimizer = SGD()#Adam()
model = create_model()
model.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [43]:
history = model.fit_generator(train_data_generator,
                             steps_per_epoch=steps_per_epoch,
                             epochs=50,
                             verbose=1,
                             validation_data=val_data_generator,
                             validation_steps=validation_steps,
                             class_weight=class_weights,
                             callbacks=callbacks)

Epoch 1/50
41/41 [==============================] - 131s 3s/step - loss: 0.5092 - acc: 0.7571 - val_loss: 0.7348 - val_acc: 0.6250
Epoch 2/50
41/41 [==============================] - 92s 2s/step - loss: 0.4126 - acc: 0.8242 - val_loss: 0.8631 - val_acc: 0.3125
Epoch 3/50
41/41 [==============================] - 97s 2s/step - loss: 0.3568 - acc: 0.8533 - val_loss: 0.7300 - val_acc: 0.5625
Epoch 4/50
41/41 [==============================] - 85s 2s/step - loss: 0.3199 - acc: 0.8702 - val_loss: 0.7546 - val_acc: 0.5625
Epoch 5/50
41/41 [==============================] - 93s 2s/step - loss: 0.3008 - acc: 0.8781 - val_loss: 0.6455 - val_acc: 0.6250
Epoch 6/50
41/41 [==============================] - 127s 3s/step - loss: 0.2833 - acc: 0.8871 - val_loss: 0.6178 - val_acc: 0.8125
Epoch 7/50
41/41 [==============================] - 120s 3s/step - loss: 0.2647 - acc: 0.8944 - val_loss: 0.7144 - val_acc: 0.5625
Epoch 8/50
41/41 [==============================] - 107s 3s/step - loss: 0.2649 - acc: 

KeyboardInterrupt: 

In [20]:
result  = model.evaluate_generator(test_data_generator, steps=len(test_data_generator), verbose=1)

1/1 [==============================] - 8s 8s/step - loss: 1.1812 - acc: 0.7051
